# S6. App Structure

## Алгоритм

1. На входе данные за 1 час - показания по экологии и деперсонализированные события по авто
2. Каждый автомобиль получает или теряет баллы в зависимости от показаний экодатчиков в момент проезда
3. Математика начисления баллов (и выставления порогов) модернизируется с каждой итерацией
4. Авто, чьи баллы превысили порог, передаются на обработку экополиции, те возвращаются к нам с информацией о том, какая достигнута точность
5. Повторяем пока не получим нужную точность

## 1. Получение входных данных

1. Отсортировать данные по времени
2. Распределить по локациям
3. Сформировать временные фреймы
4. Упаковать по временным фреймам данные по авто и экологии


In [2]:
import math
import pandas as pd
from functools import reduce

eco_df = pd.read_csv("ecodata2.csv")
car_df = pd.read_csv("cardata2.csv")

eco_df.sort_values(by=['time'], inplace=True)
car_df.sort_values(by=['time'], inplace=True)

In [3]:
eco_df.head(2)

,id,city_id,location_id,camera_id,co,no2,so2,o3,pm25,pm10,temp,hum,time,ver,lat,lon,created
4999,18,2,741,4553,4.0,0.68,0,0.81,40,45,18.24,21.28,12/10/22 17:41,-1,43.237604,76.934758,12/10/22 11:41
4858,15,2,2,420,8.8,0.00,0,0.87,35,38,19.20,19.29,12/10/22 17:41,-1,43.238362,76.889989,12/10/22 11:41


In [4]:
def createTimeframe(time, camera_id):
    eco_df_part = eco_df.loc[(pd.to_datetime(eco_df['time']) == time) & (eco_df["camera_id"] == camera_id)]
    car_df_part = car_df.loc[(pd.to_datetime(car_df['time']) == time) & (car_df["camera_id"] == camera_id)]
    return [eco_df_part, car_df_part]

In [5]:
timeframes = dict()

In [6]:
def generator_time_camera_id():
    times = eco_df['time'].unique()
    cam_ids = eco_df['camera_id'].unique()
    
    for time in times:
        for cam_id in cam_ids:
            yield (time, cam_id)

In [7]:
for time, camera_id in generator_time_camera_id():
    if not camera_id in timeframes:
        timeframes[camera_id] = {
            time: createTimeframe(time, camera_id)
        }
    else:
        timeframes[camera_id][time] = createTimeframe(time, camera_id)

In [21]:
# timeframes

## 2. Начисление баллов

1. Сформировать хранилище для баллов
2. Подготовить функцию для начисления баллов
3. Прогнать входные данные через функцию начисления баллов по каждой локации фрейм за фреймом

In [8]:
scores = {'default' : 0}

def get_pm_expect():
    pm_expect = {}
    cam_ids = eco_df["camera_id"].unique()
    
    for cam_id in cam_ids:
        pm = eco_df[eco_df["camera_id"] == cam_id]["pm25"]
        pm_avg = sum(pm) / len(pm)
        pm_sq = math.sqrt(reduce(lambda a, b: a + (b - pm_avg)**2, pm, 0) / len(pm))
        pm_expect[cam_id] = pm_sq + pm_avg
        
    return pm_expect

pm_expect = get_pm_expect()

def calc_score(eco_part_df, car_part_df, pm_exp):
    car_ids = car_part_df["id"]
    pm = eco_part_df["pm25"]
    plus = 1
    
    if pm.empty:
        return
    elif max(pm) < pm_exp: 
        plus = -1

    for car_id in car_ids:
        if not car_id in scores:
            scores[car_id] = 0
        scores[car_id] += plus
    
for (camera_id, by_camera) in timeframes.items():
    for (time, by_time) in by_camera.items():
        calc_score(by_time[0], by_time[1], pm_expect[camera_id])

1
3772
1228


## 3. Сохранение результата

Сохранить итоговое значение score в файл формата .csv

In [9]:
scores_df = pd.DataFrame(scores.values(),
                         columns=["score"],
                         index=scores.keys()
                        )

In [10]:
scores_df.to_csv('car_scores_kalmat_kenesary.csv')

In [35]:
scores

{'default': 0,
 '42e4837a-3202-030a-a7e4-11a7d4b8f6d2': 1,
 '78428b6c-0018-bcdf-f867-646d26c51a1b': 1,
 '1cb5043b-7a20-022c-389c-b8fad58534f1': 1,
 '49077b6f-908d-cf2f-20da-56d0ef85d956': 1,
 '8fce805d-09e9-6298-2ac1-68272da36dde': 1,
 '41c49918-db17-d87a-f674-7250bcecc0d3': 1,
 '042026d9-a968-1f0c-d48e-dace010c5f7e': 1,
 '7d68ac35-5ade-9eb2-fb03-f7b9fff0d794': 1,
 '679c114a-64a8-f589-adf2-0e5cf59e48e6': 1,
 '456e080d-c041-85b1-fe34-c73cc913c2ac': 1,
 'afb519f0-11ed-ec06-e618-ade776d0eff0': 1,
 '9cb6ec4d-7373-e207-2b3b-7cd97fc7675c': 1,
 '06cd58fa-28e3-685a-1f4f-8d1081346567': 1,
 '51775ea7-5929-07ec-e956-9c3618aad23e': 1,
 'dfa1cd9b-b299-5b25-5732-9aaa069336eb': 1,
 '5e200595-bb80-66f7-9d71-a2cc0b5d8b6f': 1,
 'bfd20b04-80e3-ffc5-6ac1-acdea190ff9e': 1,
 'a53f87a1-a1ab-d109-07c4-55ac9bd111bd': 1,
 'dabbf507-4b91-3f41-945d-d23ca981aa82': 1,
 'e56b5ebe-389f-475b-1bbb-9a9b512eed1d': 1,
 'ec310efb-260e-bdd8-32a7-40b4798c9a35': 1,
 'b38d8b1d-720b-303b-93d8-7fc88248208b': 1,
 'f6215c86-53ef-b